# CS-GY 9223-E: Deep Learning Homework 1
Due on Sunday, 11th February 2018, 11:55 PM

This homework can be done in pairs. Everyone must submit on NYU Classes individually.

Write down the UNIs (NetIDs) of your group (if applicable)

Member 1: John Martinez, jzm218

Member 2: Shikhar Sakhuja, ss9131

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import os
import sys
# you shouldn't need to make any more imports

In [77]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        self.parameters = {} # To be a dictionary of random weights, zero biases 
        self.num_layers = len(layer_dimensions)
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        # init parameters
        
        for l in range(1, self.num_layers):
            
            weights = np.random.randn(layer_dimensions[l], layer_dimensions[l-1]) * 0.01
            biases = np.zeros((layer_dimensions[l], 1)) 
            
            #print(weights.shape)
            
            # Creating W1, b1, W2, b2...Wl, bl
            
            self.parameters['W'+str(l)] = weights
            self.parameters['b'+str(l)] = biases
        

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        cache = {'W':W, 
                 'A':A, 
                 'b':b}
        
        return np.matrix(W) * np.matrix(A) + b, cache
        

    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        return self.relu(A)


    def relu(self, X):
        return np.maximum(0, X)
            
    def dropout(self, A, prob):
        """
        :param A: Activation
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.rand(A.shape[0], A.shape[1])
        M = (M > prob) * 1.0
        M /= 1 - prob
        A *= M
        
        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = []
        L = len(self.parameters) // 2
        Aprev = X
        AL = None
        
        for layer in range(1,L+1):
            
            """
            A1 = g1(W1, X)
            A2 = g2(W2, A1)
            
            Each cache item is a dict {W,A,b}
            
            cache[i] contains:
                {W: W(i+1)
                 A: A(i)
                 b: b(i+1)}
                 M: M(i+1)
                 AL: A(i+1) after dropout
            """
            
            W = self.parameters['W'+str(layer)] 
            b = self.parameters['b'+str(layer)]
            
            print("W in FP is:",W.shape)
            #print("Aprev is",Aprev.shape)
            #print("W is",W.shape)
            
            Z, cacheItem = self.affineForward(Aprev, W, b)  
            AL = self.activationForward(Z)
            cacheItem['Z'] = Z
            if self.drop_prob > 0 and layer < L-1:
                AL, M = dropout(AL, drop_prob) 
                cacheItem['M'] = M
                cacheItem['AL'] = AL
                
            
            Aprev = AL
            cache.append(cacheItem)
            
    
        return AL, cache
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        m = y.shape[0]
        
        #print("m in cost is:", m)
        
        AL[AL == 0] = 1e-12 # So there's no division by zero 
     
        cost = (-1 / m) * np.sum(np.multiply(y, np.log(AL)) + np.multiply(1 - y, np.log(1 - AL)))
    
        print("cost in costFunction is:",cost)
        
        if self.reg_lambda > 0:
            # add regularization
            RW = 0
            for l in range(1, self.num_layers):
                RW += np.sum(np.square(self.parameters['W'+str(l)]))
            cost += reg_lambda * RW
       
        
        # gradient of cost
        dAL = (np.divide(y, AL) - np.divide(1 - y, 1 - AL))
        
        #print("dAL in cost is:",dAL)
        
        return cost, dAL

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """        
        W, A, b = cache['W'], cache['A'], cache['b']
        dA = np.dot(W.T, dA_prev)
        dW = np.dot(dA_prev, A.T)
        print("shape of DA_prev", dA_prev.shape)
        db = np.sum(dA_prev.T)
        print("db and shape:", db, db.shape)
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        #### Potentially takes in linear activation backward cache ####
        """
        Z = cache['Z']
        dZ = np.array(dA, copy='True')
        
        if activation == 'relu':
            dZ[Z<=0] = 0
        
        #assert(dZ.shape == cache.shape)
        
        return dZ        

        
    def relu_derivative(self, dx, cached_x):
        dx = cached_x
        dx[dx <= 0] = 0
        dx[dx > 0] = 1
        return dx

    def dropout_backward(self, dA, cache):
        M = cache['M']
        dA = dA * M
        return dA

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        L = len(self.parameters) // 2
        gradients = {}
        dA_prev = dAL
        
        #print("Cache is of len",len(cache))
        
        for layer in range(L-1,-1,-1):
            
            print('index is', layer)
            
            cacheItem = cache[layer]
            if self.drop_prob > 0:
                #call dropout_backward
                dA_prev = dropout_backward(dA_prev, cacheItem)
                
            
            dA_prev = self.activationBackward(dA_prev, cacheItem)
            dA, dW, db = self.affineBackward(dA_prev, cacheItem)
            dA_prev = dA
            
            print("Gradient shape is", dW.shape)
            
            gradients["dW" + str(layer)] = dW
            gradients["db" + str(layer)] = db
            
           
            
        if self.reg_lambda > 0:
            m = Y.shape[0]
            # add gradients from L2 regularization to each dW
            for layer in range(1, num_layers+1):
                print("Layer #", layer)
                gradients['dW'+str(layer-1)] += (reg_lamba * self.parameters['W'+str(layer)]) / m
                
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        L = len(self.parameters) // 2
        
        for layer in range(1, L+1):
            param_l = str(layer)
            grad_l = str(layer-1)
            dW = gradients['dW'+grad_l]
            #print("Layer #", layer)
            #print("dW is",dW.shape,dW.T.shape)
            #print("Alpha is, ", alpha)
            #print("Weight in parameters", self.parameters['W'+param_l].shape)
            #print("Keys in parameters", self.parameters.keys())
            #print("Keys in gradients", gradients.keys())
            
            self.parameters['W'+param_l] -= alpha * gradients['dW'+grad_l] 
            #Gradients weights start with dW0, while parameters have W1.
            
            #print("Bias [parameters] shape", self.parameters['b'+param_l].shape)
            #print("Bias [parameters]", self.parameters['b'+l])
            #print("Bias [gradients] shape", gradients['db'+grad_l].shape)
            #print("Bias [gradients]", gradients['db'+str(int(l)-1)])
            self.parameters['b'+param_l] -= alpha * gradients['db'+grad_l]

    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        L = len(self.parameters) // 2
        
        for i in range(0, iters):
            
            print("THIS IS A TRAIN ITERATION")
            
            # get minibatch
            X_batch, y_batch = self.get_batch(X, y, batch_size)
            # forward prop
            AL, cache = self.forwardPropagation(X_batch)
            
            print("AL is",AL)
            print("y is", y_batch)
            print("Pred is ",self.predict(X_batch))
            
        
            
            # compute loss
            cost, dAL = self.costFunction(AL, y_batch)
            
            # compute gradients
            gradients = self.backPropagation(dAL, y_batch, cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)
            
            if i % print_every == 0:
                print("Cost: ", cost)
                print("Y is:", y_batch)
                # print cost, train and validation set accuracies
                
                
                
                
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        AL, _ = self.forwardPropagation(X)
        ALs = np.exp(AL) / np.sum(np.exp(AL), axis = 0)
        y_pred = np.argmax(ALs, axis = 0)
        return y_pred

    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        # Random indices for the samples
        indices = np.random.randint(y.shape[0]-1, size= batch_size)
        
        X_batch = X[:, indices]
        y_batch = y[indices]
        
        return X_batch, y_batch

In [10]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [4]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    One-hot encoding converts categorical labels to binary values
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    print(os.listdir())
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1 
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [5]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

['HW1-data.tar', 'cifar10-hw1', '.ipynb_checkpoints', 'HW1-uni.ipynb', 'README.md', '.git']
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [78]:
layer_dimensions = [X_train.shape[0],
1536,
768,
10]


# [i for i in range(X_train.shape[0], 9, -1)]  # including the input and output layers
#print(len(X_train))
NN = NeuralNetwork(layer_dimensions)
print("INIT DONE.")
NN.train(X_train, y_train, iters=1000, alpha=0.0001, batch_size=3, print_every=1)

INIT DONE.
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  7.89678017e-05   0.00000000e+00   0.00000000e+00]
 [  4.28517068e-06   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.47153360e-04   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.87472528e-04   1.87460663e-04   3.13228856e-04]
 [  1.06587143e-04   0.00000000e+00   0.00000000e+00]]
y is [5 9 5]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[8 8 8]]
cost in costFunction is: 1510.55203405
index is 2
shape of DA_prev (10, 3)
db and shape: 1401695.16419 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 15884.7225777 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev

W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [5 6 6]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 1565.75786324
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1565.75786324
Y is: [5 6 6]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [9 4 7]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 1842.0

index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1473.65445952
Y is: [9 0 7]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [2 1 6]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 828.930633478
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  828.930633478
Y is: [2 1 6]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0

db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1381.5510558
Y is: [8 5 2]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [4 4 7]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 1381.5510558
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1381.5510558
Y is: [4 4 7]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP

shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1105.24084464
Y is: [2 1 9]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [0 3 2]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 460.517018599
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  460.517018599
Y is: [0 3 2]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]


...

Y is: [6 0 3]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [9 8 3]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 1842.0680744
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1842.0680744
Y is: [9 8 3]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [9 4 5]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is

Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1473.65445952
Y is: [5 7 4]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [5 7 0]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 1105.24084464
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1105.24084464
Y is: [5 7 0]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL i

shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1749.96467068
Y is: [5 8 6]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [9 2 1]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 1105.24084464
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1105.24084464
Y is: [9 2 1]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)


W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 644.723826038
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  644.723826038
Y is: [2 0 5]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [8 5 9]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 2026.27488183
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gr

db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  921.034037198
Y is: [5 3 2]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
y is [0 6 5]
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
Pred is  [[0 0 0]]
cost in costFunction is: 1013.13744092
index is 2
shape of DA_prev (10, 3)
db and shape: 0.0 ()
Gradient shape is (10, 16)
index is 1
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 16)
index is 0
shape of DA_prev (16, 3)
db and shape: 0.0 ()
Gradient shape is (16, 3072)
Cost:  1013.13744092
Y is: [0 6 5]
THIS IS A TRAIN ITERATION
W in FP is: (16, 3072)
W in FP is: (16, 16)
W in FP is: (10, 16)
AL is [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 

In [ ]:
y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')